In [ ]:
# 利用 wget 命令,根据下载地址获取数据
url = "http://data.cityofdenton.com/dataset/17695047-0aeb-46a2-a9db-66847743ed1c/resource/d356a409-6764-46d7-942d-4d5a7ffb1c28/download/crime_data_20190301.csv"
!wget {url}

In [ ]:
import pandas as pd
#把数据读入，并且存入到 df 变量里面。
df = pd.read_csv('crime_data_20190301.csv')

In [ ]:
#看看 df 的前几行。
df.head()

着重分析一下，都有哪些犯罪类型，每种类型下，又有多少记录。

In [ ]:
# Pandas 中的 value_counts 函数。
# 它可以帮助我们自动统计某一列中不同类别出现的次数，而且还自动进行排序。为了显示的方便，我们只要求展示前10项内容。
df.crime.value_counts().iloc[:10]

In [ ]:
# 为了更直观查看数据统计结果，我们调用 Pandas 内置的绘图函数 plot ，并且指定绘图类型为“横向条状图”（barh）。
%matplotlib inline
df.crime.value_counts().iloc[:10].sort_values().plot(kind='barh')

In [ ]:
# 着重了解抢劫类型的情况。
robbery = df[df.crime.str.contains('ROBBERY')]; robbery.head()

In [ ]:
#robbery 数据框的大小。
robbery.shape

In [ ]:
# 使用 groupby 函数，先把犯罪位置进行分类，然后用 size 函数来查看条目统计。从大到小。
robbery.groupby('locname').size().sort_values(ascending=False)
# 根据结果显示，入室抢劫次数最多，在学校、公交车上发生的次数最少。

In [ ]:
# 用 plot 函数，把结果可视化呈现。
robbery.groupby('locname').size().sort_values(ascending=False).head(10).sort_values().plot(kind='barh')

In [ ]:
# 尝试把分析的粒度做得更加细致——研究一下，哪些街区比较危险。

回顾上图中，地址信息都表示为类似“19XX BRINKER RD”这样的方式。把具体地址的后两位隐藏，是为了保护受害者的隐私。

我们如果要统计某一条街道的犯罪数量，就需要把前面的数字忽略，并且按照街道名称加总。

这个处理起来，并不困难，只要用正则表达式即可。

In [ ]:
regex = r"\d+XX\s(?P<street>.*)"
subst = "\\g<street>"
# 这里，我们用括号把需要保留的内容，赋值为 street 分组。
# 然后替换的时候，只保留这个分组的信息。于是前面的具体地址数字就忽略了。

In [ ]:
#调用 Pandas 的 str.replace 函数，我们可以让它自动将每一个地址都进行解析替换，
#并且把结果存入到了一个新的列名称，即 street 。
robbery["street"] = robbery.publicadress.str.replace(regex, subst)

In [ ]:
robbery.head()

In [ ]:
#依然按照前面的方法，我们分组统计每一条街道上的犯罪数量，并且进行排序。
robbery.groupby('street').size().sort_values(ascending=False).head(10)

In [ ]:
# 利用时间数据，进行切分，我们就得把日期信息做一下转换处理。
!pip install python-dateutil
from dateutil.parser import *

In [ ]:
# 抽取年度信息。因为目前的日期时间列（incidentdatetime）是个字符串，
# 因此我们可以直接用 parse 函数解析它，并且抽取其中的年份（year）项。
robbery["year"] = robbery.incidentdatetime.apply(lambda x: parse(x).year)
robbery["month"] = robbery.incidentdatetime.apply(lambda x: parse(x).month)
robbery["hour"] = robbery.incidentdatetime.apply(lambda x: parse(x).hour)

In [ ]:
robbery.head()

In [ ]:
# 先按照年度来看看抢劫犯罪数量的变化趋势。
robbery.groupby('year').size()

注意这里，数量最少的是 2019 年。看似是很喜人的变化。可惜我们分析数据的时候，一定要留心这种细节。

我们读取的数据，统计时间截止到 2019 年的 3 月初。因此，2019年数据并不全。

In [ ]:
# 所以，比较稳妥的方法，是干脆去掉所有2019年的条目。
robbery = robbery[~(robbery.year == 2019)]

In [ ]:
#去除后，看看此时的 robbery 数据框。
robbery.shape
#数量没错，恰好少了 17 行。

In [ ]:
# 绘制一下抢劫犯罪数量变化趋势折线图。
robbery.groupby('year').size().plot()

In [ ]:
# 比较一下，不同月份之间，是否有明显的抢劫犯罪发生数量差别。
robbery.groupby('month').size().plot(kind='bar')

In [ ]:
# 但是，我们可能更加关心近年的情况。因为扔掉了2019年的不完整数据，我们用2018年的月份犯罪记录统计做可视化。
robbery[robbery.year==2018].groupby('month').size().plot(kind='bar')

In [ ]:
# 抢劫一般发生在什么时间。这次我们用的，是小时（hour）数据。
robbery.groupby('hour').size().plot(kind='bar')

In [ ]:
#2018年小时（hour）数据的情况。
robbery[robbery.year==2018].groupby('hour').size().plot(kind='bar')

In [ ]:
# 先按照月份分组，再按照小时分组。
robbery[robbery.year==2018].groupby(['month', 'hour']).size()

In [ ]:
# 但是这样的统计结果，无法直接绘制。我们需要做一个变换。
# 这里用的是 Pandas 中的 unstack 函数，把内侧的分组索引（hour）转换到列上。
robbery[robbery.year==2018].groupby(['month', 'hour']).size().unstack(0)

In [ ]:
# 因为许多时间段，本来就没有抢劫案件发生，所以这个表中，出现了许多空值（NaN）。
# 我们根据具体情况，采用0来填充。Pandas 中数据填充的函数是 fillna。
robbery[robbery.year==2018].groupby(['month', 'hour']).size().unstack(0).fillna(0)

我们希望绘制的，不是一张图，而是 12 张。分别代表 12 个月。这种图形，有个专门的名称，叫做“分面图”（facet plot）。 Pandas 的 plot 函数有一个非常方便的参数，叫做 subplots ，可以帮助我们轻松达成目标。\
每张图，我们依然采用柱状图的方式。因为默认方式绘制的图像，尺寸可能不符合我们的预期。因此我们显式指定图片的长宽。

In [ ]:
robbery[robbery.year==2018].groupby(['month', 'hour']).size().unstack(0).fillna(0).plot(subplots=True, kind='bar', figsize=(5,30))